In [1]:
import numpy as np
import matplotlib.pyplot as plt
# display plots in this notebook
%matplotlib inline

 #设置默认显示参数
plt.rcParams['figure.figsize'] = (10, 10)        # 图像显示大小
plt.rcParams['image.interpolation'] = 'nearest'  # 最近邻差值: 像素为正方形
plt.rcParams['image.cmap'] = 'gray'  # 使用灰度输出而不是彩色输出

In [2]:
import sys
caffe_root = '/Users/caishaofan/Desktop/BUAA/Caffe/caffe/'
print caffe_root
sys.path.insert(0, caffe_root + 'python')

import caffe

from pylab import *
%matplotlib inline
import tempfile

/Users/caishaofan/Desktop/BUAA/Caffe/caffe/


In [4]:
import scipy.io as sio
import os
import sys


## Read PIE dataset
def read_mat ( path ):
    c = 0
    X ,y ,isTest= [], [],[]

    for filename in os.listdir ( path ):
        if (filename != 'Thumbs.db'):
            data = sio.loadmat( os.path.join ( path , filename ))
#             np.concatenate((X, data['fea']),axis = 0)
#             np.concatenate((y, data['gnd']),axis = 0)
            if len(X) == 0 and len(y) == 0:
                X = data['fea']
                y = data['gnd']
                isTest = data['isTest']
            else:
                X = np.concatenate((data['fea'], X), axis = 0)
                y =np.concatenate((data['gnd'], y), axis = 0)
                isTest = np.concatenate((data['isTest'],isTest), axis = 0)
    return X, y, isTest

ROOT = 'dataset/PIE dataset/'
X,y,isTest = read_mat(ROOT)
X = np.array(X)
y = np.array(y)
isTest = np.array(isTest)

print X.shape
print X
print y.shape
print y
print isTest.shape
print isTest

(11554, 4096)
[[ 46  52  48 ..., 226 234 238]
 [ 50  39  32 ..., 255 254 254]
 [ 47  47  39 ..., 254 254 254]
 ..., 
 [ 11  10   8 ...,  36  40  42]
 [  9  11   9 ...,  29  31  36]
 [ 12   9   9 ...,  29  34  38]]
(11554, 1)
[[ 1]
 [ 1]
 [ 1]
 ..., 
 [68]
 [68]
 [68]]
(11554, 1)
[[ 0.]
 [ 0.]
 [ 0.]
 ..., 
 [ 1.]
 [ 0.]
 [ 0.]]


In [5]:
X_train, X_test, y_train, y_test = [],[],[],[]
for i in np.arange(len(isTest)):
    if(isTest[i] == 0):
        if len(X_train) == 0 and len(y_train) == 0:
            X_train = X[i]
            y_train = y[i]
        else:
            X_train = np.vstack((X[i], X_train))
            y_train = np.vstack((y[i], y_train))
    else:
        if len(X_test) == 0 and len(y_test) == 0:
            X_test = X[i]
            y_test = y[i]
        else:
            X_test = np.vstack((X[i], X_test))
            y_test = np.vstack((y[i], y_test))

print X_train
print X_train.shape

[]
[]
[[ 12   9   9 ...,  29  34  38]
 [  9  11   9 ...,  29  31  36]
 [  8  10  12 ...,  25  25  22]
 ..., 
 [ 47  47  39 ..., 254 254 254]
 [ 50  39  32 ..., 255 254 254]
 [ 46  52  48 ..., 226 234 238]]
(10262, 4096)


In [6]:
# show the shape of X_train X_test y_train y_test
print X_train
print X_train.shape
print X_test
print X_test.shape

y_train = y_train.reshape(y_train.shape[0])
y_test = y_test.reshape(y_test.shape[0])
print y_train.shape
print y_test.shape

[[ 12   9   9 ...,  29  34  38]
 [  9  11   9 ...,  29  31  36]
 [  8  10  12 ...,  25  25  22]
 ..., 
 [ 47  47  39 ..., 254 254 254]
 [ 50  39  32 ..., 255 254 254]
 [ 46  52  48 ..., 226 234 238]]
(10262, 4096)
[[ 11  10   8 ...,  36  40  42]
 [ 10  11  21 ...,  36  36  34]
 [  4   4   5 ...,  26  23  20]
 ..., 
 [ 22  21  26 ...,  48  47  44]
 [ 16  16  17 ...,  25  25  24]
 [ 98 195 181 ...,  81  76  72]]
(1292, 4096)
(10262,)
(1292,)


In [8]:
weights = os.path.join(caffe_root, 'models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel')
assert os.path.exists(weights)


In [17]:
from caffe import layers as L
from caffe import params as P

weight_params = dict(lr_mult = 1, decay_mult = 1)
bias_params = dict(lr_mult = 2, decay_mult = 0)
learned_param = [weight_params, bias_params]

frozen_param = [dict(lr_mult=0)] * 2

def conv_relu(bottom, ks, nout, stride=1, pad=0, group=1,
              param=learned_param,
              weight_filler=dict(type='gaussian', std=0.01),
              bias_filler=dict(type='constant', value=0.1)):
    conv = L.Convolution(bottom, kernel_size=ks, stride=stride,
                         num_output=nout, pad=pad, group=group,
                         param=param, weight_filler=weight_filler,
                         bias_filler=bias_filler)
    return conv, L.ReLU(conv, in_place=True)

def fc_relu(bottom, nout, param=learned_param,
            weight_filler=dict(type='gaussian', std=0.005),
            bias_filler=dict(type='constant', value=0.1)):
    fc = L.InnerProduct(bottom, num_output=nout, param=param,
                        weight_filler=weight_filler,
                        bias_filler=bias_filler)
    return fc, L.ReLU(fc, in_place=True)

def max_pool(bottom, ks, stride=1):
    return L.Pooling(bottom, pool=P.Pooling.MAX, kernel_size=ks, stride=stride)

def caffenet(data, label=None, train=True, num_classes=1000,
             classifier_name='fc8', learn_all=False):
    """Returns a NetSpec specifying CaffeNet, following the original proto text
       specification (./models/bvlc_reference_caffenet/train_val.prototxt)."""
    n = caffe.NetSpec()
    n.data = data
    param = learned_param if learn_all else frozen_param
    n.conv1, n.relu1 = conv_relu(n.data, 11, 96, stride=4, param=param)
    n.pool1 = max_pool(n.relu1, 3, stride=2)
    n.norm1 = L.LRN(n.pool1, local_size=5, alpha=1e-4, beta=0.75)
    n.conv2, n.relu2 = conv_relu(n.norm1, 5, 256, pad=2, group=2, param=param)
    n.pool2 = max_pool(n.relu2, 3, stride=2)
    n.norm2 = L.LRN(n.pool2, local_size=5, alpha=1e-4, beta=0.75)
    n.conv3, n.relu3 = conv_relu(n.norm2, 3, 384, pad=1, param=param)
    n.conv4, n.relu4 = conv_relu(n.relu3, 3, 384, pad=1, group=2, param=param)
    n.conv5, n.relu5 = conv_relu(n.relu4, 3, 256, pad=1, group=2, param=param)
    n.pool5 = max_pool(n.relu5, 3, stride=2)
    n.fc6, n.relu6 = fc_relu(n.pool5, 4096, param=param)
    if train:
        n.drop6 = fc7input = L.Dropout(n.relu6, in_place=True)
    else:
        fc7input = n.relu6
    n.fc7, n.relu7 = fc_relu(fc7input, 4096, param=param)
    if train:
        n.drop7 = fc8input = L.Dropout(n.relu7, in_place=True)
    else:
        fc8input = n.relu7
    # always learn fc8 (param=learned_param)
    fc8 = L.InnerProduct(fc8input, num_output=num_classes, param=learned_param)
    # give fc8 the name specified by argument `classifier_name`
    n.__setattr__(classifier_name, fc8)
    if not train:
        n.probs = L.Softmax(fc8)
    if label is not None:
        n.label = label
        n.loss = L.SoftmaxWithLoss(fc8, n.label)
        n.acc = L.Accuracy(fc8, n.label)
    # write the net to a temporary file and return its filename
    with tempfile.NamedTemporaryFile(delete=False) as f:
        f.write(str(n.to_proto()))
        return f.name

In [18]:
from pylab import *
import tempfile
dummy_data = L.DummyData(shape=dict(dim=[1, 3, 227, 227]))
imagenet_net_filename = caffenet(data=dummy_data, train=False)
imagenet_net = caffe.Net(imagenet_net_filename, weights, caffe.TEST)


In [ ]:
def style_net(train=True, learn_all=False, subset=None):
    if subset is None:
        subset = 'train' if train else 'test'
    source = caffe_root + 'data/flickr_style/%s.txt' % subset
    transform_param = dict(mirror=train, crop_size=227,
        mean_file=caffe_root + 'data/ilsvrc12/imagenet_mean.binaryproto')
    style_data, style_label = L.ImageData(
        transform_param=transform_param, source=source,
        batch_size=50, new_height=256, new_width=256, ntop=2)
    return caffenet(data=style_data, label=style_label, train=train,
                    num_classes=NUM_STYLE_LABELS,
                    classifier_name='fc8_flickr',
                    learn_all=learn_all)

In [3]:
# Download just a small subset of the data for this exercise.
# (2000 of 80K images, 5 of 20 labels.)
# To download the entire dataset, set `full_dataset = True`.
full_dataset = False
if full_dataset:
    NUM_STYLE_IMAGES = NUM_STYLE_LABELS = -1
else:
    NUM_STYLE_IMAGES = 2000
    NUM_STYLE_LABELS = 5

# This downloads the ilsvrc auxiliary data (mean file, etc),
# and a subset of 2000 images for the style recognition task.
import os
os.chdir(caffe_root)  # run scripts from caffe root
!python examples/finetune_flickr_style/assemble_data.py \
    --workers=-1  --seed=1701 \
    --images=$NUM_STYLE_IMAGES  --label=$NUM_STYLE_LABELS
# back to examples
os.chdir('examples')

Writing train/val for 0 successfully downloaded images.
